In [5]:
import cv2, numpy as np
from skimage import filters, restoration, morphology, exposure
from PIL import Image

def load_gray(path):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    if img is None: raise FileNotFoundError(path)
    return img

def deskew(img):
    # compute skew via Hough on edges
    edges = cv2.Canny(img, 50, 150)
    lines = cv2.HoughLines(edges, 1, np.pi/180, 150)
    if lines is None: return img
    angles = []
    for rho, theta in lines[:,0]:
        angle = (theta*180/np.pi) - 90
        if -30 < angle < 30:
            angles.append(angle)
    if not angles: return img
    angle = np.median(angles)
    (h,w) = img.shape
    M = cv2.getRotationMatrix2D((w//2,h//2), angle, 1.0)
    return cv2.warpAffine(img, M, (w,h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

def enhance_to_mask(img):
    # smooth noise but preserve edges
    den = cv2.bilateralFilter(img, d=7, sigmaColor=35, sigmaSpace=35)
    # increase contrast
    den = exposure.rescale_intensity(den)
    # adaptive threshold (Sauvola works great on paper)
    thr = filters.threshold_sauvola(den, window_size=41, k=0.2)
    mask = (den < thr).astype(np.uint8)  # signature as 1s
    # remove small specks, close gaps
    mask = morphology.remove_small_objects(mask.astype(bool), min_size=40)
    mask = morphology.binary_closing(mask, morphology.disk(1))
    #mask = morphology.thin(mask)  # optional slight thinning
    return (mask*255).astype(np.uint8)

def unsharp(img):
    blur = cv2.GaussianBlur(img, (0,0), sigmaX=1.0)
    sharp = cv2.addWeighted(img, 1.6, blur, -0.6, 0)
    return sharp

def save_transparent(signature_mask, out_png_path, stroke_color=(0,0,0)):
    # mask is white where ink; create RGBA with transparent bg
    h,w = signature_mask.shape
    rgba = np.zeros((h,w,4), dtype=np.uint8)
    rgba[..., :3] = stroke_color  # black
    rgba[..., 3] = signature_mask  # alpha from mask
    Image.fromarray(rgba, mode='RGBA').save(out_png_path)

def process(in_path, out_path="signature_clean.png"):
    img = load_gray(in_path)
    img = deskew(img)
    mask = enhance_to_mask(img)
    mask = unsharp(mask)
    save_transparent(mask, out_path)
    print(f"Saved: {out_path}")

process("IMG_1675.jpg", "signature_clean.png")


Saved: signature_clean.png


/tmp/ipykernel_3951/1307290735.py:51: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  Image.fromarray(rgba, mode='RGBA').save(out_png_path)
